In [30]:
import pandas as pd
import numpy as np
import itertools
import datetime
import pandas_gbq
import matplotlib.pyplot as plt
from datetime import *
from datetime import datetime, timedelta, date
# %load_ext google.colab.data_table
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Standard plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
#Race Results

In [4]:
#Change the date to the previous day
query = """
SELECT
*
FROM
`perceptive-ivy-290216.f1_api.results_race`
# WHERE Year=2024 AND GP="Austrian Grand Prix"
"""
project_id = "perceptive-ivy-290216"
df_bq2 = pandas_gbq.read_gbq(query, project_id=project_id, dialect='standard')


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/google/cloud/bigquery/table.py:2379: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


In [5]:
GP=df_bq2
GP=GP.fillna(0)
GP["Position"]=GP["Position"].astype(int)
GP["GridPosition"]=GP["GridPosition"].astype(int)
GP["Points"]=GP["Points"].astype(int)
GP=GP[GP["Position"]!=0]
GP=GP[GP["Position"]<21]
GP.head()

,DriverNumber,BroadcastName,Abbreviation,DriverId,TeamName,TeamColor,TeamId,FirstName,LastName,FullName,HeadshotUrl,CountryCode,Position,ClassifiedPosition,GridPosition,Q1,Q2,Q3,Time,Status,Points,Year,GP
0,2,,,,Brabham,,0,Teo,Fabi,Teo Fabi,0,0,4,0,7,NaT,NaT,NaT,0 days 01:22:09.163000,Finished,3,1984,Austrian Grand Prix
1,2,,,,Brabham,,0,Teo,Fabi,Teo Fabi,0,0,3,0,23,NaT,NaT,NaT,0 days 01:57:08.370000,Finished,4,1984,Detroit Grand Prix
2,71,,,,Kurtis Kraft,,0,Al,Herman,Al Herman,0,0,7,0,16,NaT,NaT,NaT,0 days 04:00:23.770000,Finished,0,1955,Indianapolis 500
3,89,,,,Dunn,,0,Al,Herman,Al Herman,0,0,13,0,23,NaT,NaT,NaT,0 days 03:47:29.600000,Finished,0,1959,Indianapolis 500
4,52,,,,Kurtis Kraft,,0,Al,Keller,Al Keller,0,0,11,0,21,NaT,NaT,NaT,0 days 03:53:28,Finished,0,1958,Indianapolis 500


In [6]:
#Get the counts by final position in race

In [7]:
GP_Race_Wins=GP[GP["Year"].isin(range(1965,2024))].pivot(index=['Year','GP'], columns=['Position'], values=["FullName"])
GP_Race_Wins.head()

FullName                                 \
Position                        1               2              3    
Year GP                                                             
1965 Belgian Grand Prix  Jim Clark  Jackie Stewart  Bruce McLaren   
     British Grand Prix  Jim Clark     Graham Hill   John Surtees   
     Dutch Grand Prix    Jim Clark  Jackie Stewart     Dan Gurney   
     French Grand Prix   Jim Clark  Jackie Stewart   John Surtees   
     German Grand Prix   Jim Clark     Graham Hill     Dan Gurney   

                                                                        \
Position                           4               5                6    
Year GP                                                                  
1965 Belgian Grand Prix  Jack Brabham     Graham Hill   Richie Ginther   
     British Grand Prix   Mike Spence  Jackie Stewart       Dan Gurney   
     Dutch Grand Prix     Graham Hill     Denny Hulme   Richie Ginther   
     French Grand Prix    Denny Hulme     Graham Hill       Jo Siffert   
     German Grand Prix   Jochen Rindt    Jack Brabham  Lorenzo Bandini   

                                                                         \
Position                           7                8                9    
Year GP                                                                   
1965 Belgian Grand Prix   Mike Spence       Jo Siffert  Lorenzo Bandini   
     British Grand Prix    Jo Bonnier    Frank Gardner       Jo Siffert   
     Dutch Grand Prix    John Surtees      Mike Spence  Lorenzo Bandini   
     French Grand Prix    Mike Spence  Lorenzo Bandini     Bob Anderson   
     German Grand Prix     Jo Bonnier   Masten Gregory     John Surtees   

                                                                        \
Position                            10             11               12   
Year GP                                                                  
1965 Belgian Grand Prix     Dan Gurney   Jochen Rindt   Lucien Bianchi   
     British Grand Prix  Bruce McLaren       Ian Raby   Masten Gregory   
     Dutch Grand Prix    Innes Ireland  Frank Gardner  Richard Attwood   
     French Grand Prix   Bruce McLaren     Jo Bonnier       Chris Amon   
     German Grand Prix      Jo Siffert    Mike Spence   Gerhard Mitter   

                                                                           \
Position                              13               14              15   
Year GP                                                                     
1965 Belgian Grand Prix    Innes Ireland  Richard Attwood  Masten Gregory   
     British Grand Prix  Richard Attwood     Jochen Rindt   Innes Ireland   
     Dutch Grand Prix         Jo Siffert     Jochen Rindt   Bruce McLaren   
     French Grand Prix     Innes Ireland       Dan Gurney  Richie Ginther   
     German Grand Prix   Richard Attwood    Bruce McLaren     Denny Hulme   

                                                                         \
Position                             16              17              18   
Year GP                                                                   
1965 Belgian Grand Prix      Jo Bonnier  Ronnie Bucknum    John Surtees   
     British Grand Prix     John Rhodes    Bob Anderson     Denny Hulme   
     Dutch Grand Prix        Jo Bonnier    Bob Anderson             NaN   
     French Grand Prix   Ronnie Bucknum    Jochen Rindt             NaN   
     German Grand Prix       Chris Amon    Paul Hawkins  Jackie Stewart   

                                                             
Position                             19                  20  
Year GP                                                      
1965 Belgian Grand Prix   Frank Gardner                 NaN  
     British Grand Prix  Richie Ginther     Lorenzo Bandini  
     Dutch Grand Prix               NaN                 NaN  
     French Grand Prix              NaN                 NaN  
     German Grand Prix    Frank Ga

In [8]:
#Remove the column that says full name
GP_Race_Wins.columns = GP_Race_Wins.columns.droplevel(level=0)

#Reset the index to get a proper df
GP_Race_Wins=GP_Race_Wins.reset_index()

GP_Race_Wins=GP_Race_Wins.sort_values(by=["Year","GP"],ascending=[False,True])
GP_Race_Wins.head()

Position,Year,GP,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
948,2023,Abu Dhabi Grand Prix,Max Verstappen,Charles Leclerc,George Russell,Sergio Perez,Lando Norris,Oscar Piastri,Fernando Alonso,Yuki Tsunoda,Lewis Hamilton,Lance Stroll,Daniel Ricciardo,Esteban Ocon,Pierre Gasly,Alexander Albon,Nico Hulkenberg,Logan Sargeant,Guanyu Zhou,Carlos Sainz,Valtteri Bottas,Kevin Magnussen
949,2023,Australian Grand Prix,Max Verstappen,Lewis Hamilton,Fernando Alonso,Lance Stroll,Sergio Perez,Lando Norris,Nico Hulkenberg,Oscar Piastri,Guanyu Zhou,Yuki Tsunoda,Valtteri Bottas,Carlos Sainz,Pierre Gasly,Esteban Ocon,Nyck De Vries,Logan Sargeant,Kevin Magnussen,George Russell,Alexander Albon,Charles Leclerc
950,2023,Austrian Grand Prix,Max Verstappen,Charles Leclerc,Sergio Perez,Lando Norris,Fernando Alonso,Carlos Sainz,George Russell,Lewis Hamilton,Lance Stroll,Pierre Gasly,Alexander Albon,Guanyu Zhou,Logan Sargeant,Esteban Ocon,Valtteri Bottas,Oscar Piastri,Nyck De Vries,Kevin Magnussen,Yuki Tsunoda,Nico Hulkenberg
951,2023,Azerbaijan Grand Prix,Sergio Perez,Max Verstappen,Charles Leclerc,Fernando Alonso,Carlos Sainz,Lewis Hamilton,Lance Stroll,George Russell,Lando Norris,Yuki Tsunoda,Oscar Piastri,Alexander Albon,Kevin Magnussen,Pierre Gasly,Esteban Ocon,Logan Sargeant,Nico Hulkenberg,Valtteri Bottas,Guanyu Zhou,Nyck De Vries
952,2023,Bahrain Grand Prix,Max Verstappen,Sergio Perez,Fernando Alonso,Carlos Sainz,Lewis Hamilton,Lance Stroll,George Russell,Valtteri Bottas,Pierre Gasly,Alexander Albon,Yuki Tsunoda,Logan Sargeant,Kevin Magnussen,Nyck De Vries,Nico Hulkenberg,Guanyu Zhou,Lando Norris,Esteban Ocon,Charles Leclerc,Oscar Piastri


In [9]:
import plotly.graph_objects as go

fig = go.Figure(go.Table
 (header={"values": GP_Race_Wins.columns},
  cells=dict(
    values=GP_Race_Wins.T.values,
    # fill_color=[GP_Race_Wins.Color],
    align='center', font=dict(color='Black', size=11)
    ))
 )

# fig.update_layout(
#     updatemenus=[
#         {
#             "buttons": [
#                 {
#                     "label": c,
#                     "method": "update",
#                     "args": [
#                         {
#                             "cells": {
#                                 "values": GP_Race_Wins.T.values
#                                 if c == "All"
#                                 else GP_Race_Wins.loc[GP_Race_Wins["Year"].eq(c)].T.values
#                             }
#                         }
#                     ],
#                 }
#                 for c in ["All"] + GP_Race_Wins["Year"].unique().tolist()
#             ]
#         }
#     ]
# )

fig.update_layout(
    title="Race Results All Time By Position and Driver",
        title_x=0.5,
        template="plotly_white",

    yaxis = dict(tickfont = dict(size=12)),
    xaxis = dict(tickfont = dict(size=12)),
    font=dict(
        family="PT Sans Narrow",
        size=12,
        color="Black"
    ),
    height=900,
    width=1400,
    title_font_family="PT Sans Narrow",
    margin=dict(l=10, r=10, t=35, b=10),
)
fig.show()

In [10]:
fig.write_html("Race Results All Time By Position and Driver.html")

In [11]:
#Sprint Results

In [12]:
#Change the date to the previous day
query = """
SELECT
*
FROM
`perceptive-ivy-290216.f1_api.results_sprint`
# WHERE Year=2024 AND GP="Austrian Grand Prix"
"""
project_id = "perceptive-ivy-290216"
df_bq3 = pandas_gbq.read_gbq(query, project_id=project_id, dialect='standard')


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/google/cloud/bigquery/table.py:2379: UserWarning:

A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.



In [13]:
GP_sprint=df_bq3
GP_sprint=GP_sprint.fillna(0)
GP_sprint["Position"]=GP_sprint["Position"].astype(int)
GP_sprint["GridPosition"]=GP_sprint["GridPosition"].astype(int)
GP_sprint["Points"]=GP_sprint["Points"].astype(int)
GP_sprint=GP_sprint[GP_sprint["Position"]!=0]
GP_sprint.head()

,DriverNumber,BroadcastName,Abbreviation,DriverId,TeamName,TeamColor,TeamId,FirstName,LastName,FullName,HeadshotUrl,CountryCode,Position,ClassifiedPosition,GridPosition,Q1,Q2,Q3,Time,Status,Points,Year,GP
0,31,E OCON,OCO,OCO,Alpine,0090FF,0,Esteban,Ocon,Esteban Ocon,0,0,9,0,10,NaT,NaT,NaT,0 days 00:29:44.191000,Finished,0,2021,São Paulo Grand Prix
1,31,E OCON,OCO,OCO,Alpine,0090FF,0,Esteban,Ocon,Esteban Ocon,0,0,13,0,14,NaT,NaT,NaT,0 days 00:28:37.451000,Finished,0,2021,Italian Grand Prix
2,31,E OCON,OCO,OCO,Alpine,0090FF,0,Esteban,Ocon,Esteban Ocon,0,0,10,0,13,NaT,NaT,NaT,0 days 00:26:25.821000,Finished,0,2021,British Grand Prix
3,31,E OCON,OCO,OCO,Alpine,2293D1,0,Esteban,Ocon,Esteban Ocon,0,0,9,0,9,NaT,NaT,NaT,0 days 00:25:20.843000,Finished,0,2023,Belgian Grand Prix
4,31,E OCON,OCO,OCO,Alpine,2293D1,0,Esteban,Ocon,Esteban Ocon,0,0,7,0,8,NaT,NaT,NaT,0 days 00:31:03.332000,Finished,2,2023,Austrian Grand Prix


In [14]:
#Get the counts by final position in race

In [15]:
GP_Sprint_Wins=GP_sprint[GP_sprint["Year"].isin(range(1900,2025))].pivot(index=['Year','GP'], columns=['Position'], values=["FullName"])
GP_Sprint_Wins.head()

FullName                   \
Position                                     1                2    
Year GP                                                            
2021 British Grand Prix          Max Verstappen   Lewis Hamilton   
     Italian Grand Prix         Valtteri Bottas   Max Verstappen   
     São Paulo Grand Prix       Valtteri Bottas   Max Verstappen   
2022 Austrian Grand Prix         Max Verstappen  Charles Leclerc   
     Emilia Romagna Grand Prix   Max Verstappen  Charles Leclerc   

                                                                   \
Position                                      3                4    
Year GP                                                             
2021 British Grand Prix          Valtteri Bottas  Charles Leclerc   
     Italian Grand Prix         Daniel Ricciardo     Lando Norris   
     São Paulo Grand Prix           Carlos Sainz     Sergio Perez   
2022 Austrian Grand Prix            Carlos Sainz   George Russell   
     Emilia Romagna Grand Prix      Sergio Perez     Carlos Sainz   

                                                                  \
Position                                    5                 6    
Year GP                                                            
2021 British Grand Prix           Lando Norris  Daniel Ricciardo   
     Italian Grand Prix         Lewis Hamilton   Charles Leclerc   
     São Paulo Grand Prix       Lewis Hamilton      Lando Norris   
2022 Austrian Grand Prix          Sergio Perez      Esteban Ocon   
     Emilia Romagna Grand Prix    Lando Norris  Daniel Ricciardo   

                                                                     \
Position                                     7                   8    
Year GP                                                               
2021 British Grand Prix         Fernando Alonso    Sebastian Vettel   
     Italian Grand Prix            Carlos Sainz  Antonio Giovinazzi   
     São Paulo Grand Prix       Charles Leclerc        Pierre Gasly   
2022 Austrian Grand Prix        Kevin Magnussen      Lewis Hamilton   
     Emilia Romagna Grand Prix  Valtteri Bottas     Kevin Magnussen   

                                                                   \
Position                                     9                 10   
Year GP                                                             
2021 British Grand Prix          George Russell      Esteban Ocon   
     Italian Grand Prix            Sergio Perez      Lance Stroll   
     São Paulo Grand Prix          Esteban Ocon  Sebastian Vettel   
2022 Austrian Grand Prix        Mick Schumacher   Valtteri Bottas   
     Emilia Romagna Grand Prix  Fernando Alonso   Mick Schumacher   

                                                                    \
Position                                      11                12   
Year GP                                                              
2021 British Grand Prix             Carlos Sainz      Pierre Gasly   
     Italian Grand Prix          Fernando Alonso  Sebastian Vettel   
     São Paulo Grand Prix       Daniel Ricciardo   Fernando Alonso   
2022 Austrian Grand Prix            Lando Norris  Daniel Ricciardo   
     Emilia Romagna Grand Prix    George Russell      Yuki Tsunoda   

                                                                     \
Position                                        13               14   
Year GP                                                               
2021 British Grand Prix             Kimi Räikkönen     Lance Stroll   
     Italian Grand Prix               Esteban Ocon  Nicholas Latifi   
     São Paulo Grand Prix       Antonio Giovinazzi     Lance Stroll   
2022 Austrian Grand Prix              Lance Stroll      Guanyu Zhou   
     Emilia Romagna Grand Prix    Sebastian Vettel   Lewis Hamilton   

                                                                     \
Position                                        15               16   
Ye

In [16]:
#Remove the column that says full name
GP_Sprint_Wins.columns = GP_Sprint_Wins.columns.droplevel(level=0)

#Reset the index to get a proper df
GP_Sprint_Wins=GP_Sprint_Wins.reset_index()

GP_Sprint_Wins=GP_Sprint_Wins.sort_values(by=["Year","GP"],ascending=[False,True])
GP_Sprint_Wins.head()

Position,Year,GP,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
12,2024,Austrian Grand Prix,Max Verstappen,Oscar Piastri,Lando Norris,George Russell,Carlos Sainz,Lewis Hamilton,Charles Leclerc,Sergio Perez,Kevin Magnussen,Lance Stroll,Esteban Ocon,Pierre Gasly,Yuki Tsunoda,Daniel Ricciardo,Fernando Alonso,Logan Sargeant,Alexander Albon,Valtteri Bottas,Nico Hulkenberg,Guanyu Zhou
13,2024,Chinese Grand Prix,Max Verstappen,Lewis Hamilton,Sergio Perez,Charles Leclerc,Carlos Sainz,Lando Norris,Oscar Piastri,George Russell,Guanyu Zhou,Kevin Magnussen,Daniel Ricciardo,Valtteri Bottas,Esteban Ocon,Lance Stroll,Pierre Gasly,Yuki Tsunoda,Alexander Albon,Logan Sargeant,Nico Hulkenberg,Fernando Alonso
14,2024,Miami Grand Prix,Max Verstappen,Charles Leclerc,Sergio Perez,Daniel Ricciardo,Carlos Sainz,Oscar Piastri,Nico Hulkenberg,Yuki Tsunoda,Pierre Gasly,Logan Sargeant,Guanyu Zhou,George Russell,Alexander Albon,Valtteri Bottas,Esteban Ocon,Lewis Hamilton,Fernando Alonso,Kevin Magnussen,Lance Stroll,Lando Norris
6,2023,Austrian Grand Prix,Max Verstappen,Sergio Perez,Carlos Sainz,Lance Stroll,Fernando Alonso,Nico Hulkenberg,Esteban Ocon,George Russell,Lando Norris,Lewis Hamilton,Oscar Piastri,Charles Leclerc,Alexander Albon,Kevin Magnussen,Pierre Gasly,Yuki Tsunoda,Nyck De Vries,Logan Sargeant,Guanyu Zhou,Valtteri Bottas
7,2023,Azerbaijan Grand Prix,Sergio Perez,Charles Leclerc,Max Verstappen,George Russell,Carlos Sainz,Fernando Alonso,Lewis Hamilton,Lance Stroll,Alexander Albon,Oscar Piastri,Kevin Magnussen,Guanyu Zhou,Pierre Gasly,Nyck De Vries,Nico Hulkenberg,Valtteri Bottas,Lando Norris,Esteban Ocon,Yuki Tsunoda,NaN


In [17]:
import plotly.graph_objects as go

fig_sprint = go.Figure(go.Table
 (header={"values": GP_Sprint_Wins.columns},
  cells=dict(
    values=GP_Sprint_Wins.T.values,
    # fill_color=[GP_Sprint_Wins.Color],
    align='center', font=dict(color='Black', size=11)
    ))
 )

# fig.update_layout(
#     updatemenus=[
#         {
#             "buttons": [
#                 {
#                     "label": c,
#                     "method": "update",
#                     "args": [
#                         {
#                             "cells": {
#                                 "values": GP_Sprint_Wins.T.values
#                                 if c == "All"
#                                 else GP_Sprint_Wins.loc[GP_Sprint_Wins["Year"].eq(c)].T.values
#                             }
#                         }
#                     ],
#                 }
#                 for c in ["All"] + GP_Sprint_Wins["Year"].unique().tolist()
#             ]
#         }
#     ]
# )

fig_sprint.update_layout(
    title="Sprint Race Results All Time By Position and Driver",
        title_x=0.5,
        template="plotly_white",

    yaxis = dict(tickfont = dict(size=12)),
    xaxis = dict(tickfont = dict(size=12)),
    font=dict(
        family="PT Sans Narrow",
        size=12,
        color="Black"
    ),
    height=750,
    width=1400,
    title_font_family="PT Sans Narrow",
    margin=dict(l=10, r=10, t=35, b=10),
)
fig_sprint.show()

In [18]:
fig_sprint.write_html("Sprint Race Results All Time By Position and Driver.html")

In [19]:
#Team Race Results

In [20]:
#Get the counts by final position in race

In [21]:
GP_Race_Wins_Team=GP[GP["Year"].isin(range(2021,2025))].pivot(index=['Year','GP'], columns=['Position'], values=["TeamName"])
GP_Race_Wins_Team.head()

TeamName                               \
Position                                 1                2           3    
Year GP                                                                    
2021 Abu Dhabi Grand Prix   Red Bull Racing         Mercedes     Ferrari   
     Austrian Grand Prix    Red Bull Racing         Mercedes     McLaren   
     Azerbaijan Grand Prix  Red Bull Racing     Aston Martin  AlphaTauri   
     Bahrain Grand Prix            Mercedes  Red Bull Racing    Mercedes   
     Belgian Grand Prix     Red Bull Racing         Williams    Mercedes   

                                                                          \
Position                            4                5                6    
Year GP                                                                    
2021 Abu Dhabi Grand Prix   AlphaTauri       AlphaTauri         Mercedes   
     Austrian Grand Prix      Mercedes          Ferrari  Red Bull Racing   
     Azerbaijan Grand Prix     Ferrari          McLaren           Alpine   
     Bahrain Grand Prix        McLaren  Red Bull Racing          Ferrari   
     Belgian Grand Prix        McLaren     Aston Martin       AlphaTauri   

                                                             \
Position                            7        8           9    
Year GP                                                       
2021 Abu Dhabi Grand Prix      McLaren   Alpine      Alpine   
     Austrian Grand Prix       McLaren  Ferrari  AlphaTauri   
     Azerbaijan Grand Prix  AlphaTauri  Ferrari     McLaren   
     Bahrain Grand Prix        McLaren  Ferrari  AlphaTauri   
     Belgian Grand Prix         Alpine  Ferrari    Williams   

                                                                  \
Position                                   10                 11   
Year GP                                                            
2021 Abu Dhabi Grand Prix             Ferrari       Aston Martin   
     Austrian Grand Prix               Alpine           Williams   
     Azerbaijan Grand Prix  Alfa Romeo Racing  Alfa Romeo Racing   
     Bahrain Grand Prix          Aston Martin  Alfa Romeo Racing   
     Belgian Grand Prix               Ferrari             Alpine   

                                                                  \
Position                                   12                 13   
Year GP                                                            
2021 Abu Dhabi Grand Prix             McLaren       Aston Martin   
     Austrian Grand Prix           AlphaTauri       Aston Martin   
     Azerbaijan Grand Prix           Mercedes       Haas F1 Team   
     Bahrain Grand Prix     Alfa Romeo Racing             Alpine   
     Belgian Grand Prix              Mercedes  Alfa Romeo Racing   

                                                                  \
Position                                   14                 15   
Year GP                                                            
2021 Abu Dhabi Grand Prix        Haas F1 Team    Red Bull Racing   
     Austrian Grand Prix    Alfa Romeo Racing  Alfa Romeo Racing   
     Azerbaijan Grand Prix       Haas F1 Team           Mercedes   
     Bahrain Grand Prix              Williams       Aston Martin   
     Belgian Grand Prix               McLaren         AlphaTauri   

                                                             \
Position                              16                 17   
Year GP                                                       
2021 Abu Dhabi Grand Prix       Williams  Alfa Romeo Racing   
     Austrian Grand Prix        Williams       Aston Martin   
     Azerbaijan Grand Prix      Williams           Williams   
     Bahrain Grand Prix     Haas F1 Team         AlphaTauri   
     Belgian Grand Prix     Haas F1 Team       Haas F1 Team   

                                                                                
Position                                   18                 19            20  
Year GP   

In [22]:
#Remove the column that says full name
GP_Race_Wins_Team.columns = GP_Race_Wins_Team.columns.droplevel(level=0)

#Reset the index to get a proper df
GP_Race_Wins_Team=GP_Race_Wins_Team.reset_index()

GP_Race_Wins_Team=GP_Race_Wins_Team.sort_values(by=["Year","GP"],ascending=[False,True])
GP_Race_Wins_Team.head()

Position,Year,GP,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
66,2024,Australian Grand Prix,Ferrari,Ferrari,McLaren,McLaren,Red Bull Racing,Aston Martin,RB,Aston Martin,Haas F1 Team,Haas F1 Team,Williams,RB,Alpine,Kick Sauber,Kick Sauber,Alpine,Mercedes,Mercedes,Red Bull Racing,NaN
67,2024,Austrian Grand Prix,Mercedes,McLaren,Ferrari,Mercedes,Red Bull Racing,Haas F1 Team,Red Bull Racing,Haas F1 Team,RB,Alpine,Ferrari,Alpine,Aston Martin,RB,Williams,Kick Sauber,Kick Sauber,Aston Martin,Williams,McLaren
68,2024,Bahrain Grand Prix,Red Bull Racing,Red Bull Racing,Ferrari,Ferrari,Mercedes,McLaren,Mercedes,McLaren,Aston Martin,Aston Martin,Kick Sauber,Haas F1 Team,RB,RB,Williams,Haas F1 Team,Alpine,Alpine,Kick Sauber,Williams
69,2024,Belgian Grand Prix,Mercedes,McLaren,Ferrari,Red Bull Racing,McLaren,Ferrari,Red Bull Racing,Aston Martin,Alpine,RB,Aston Martin,Williams,Alpine,Haas F1 Team,Kick Sauber,RB,Williams,Haas F1 Team,Kick Sauber,Mercedes
70,2024,British Grand Prix,Mercedes,Red Bull Racing,McLaren,McLaren,Ferrari,Haas F1 Team,Aston Martin,Aston Martin,Williams,RB,Williams,Haas F1 Team,RB,Ferrari,Kick Sauber,Alpine,Red Bull Racing,Kick Sauber,Mercedes,Alpine


In [23]:
import plotly.graph_objects as go

fig_team = go.Figure(go.Table
 (header={"values": GP_Race_Wins_Team.columns},
  cells=dict(
    values=GP_Race_Wins_Team.T.values,
    # fill_color=[GP_Race_Wins_Team.Color],
    align='center', font=dict(color='Black', size=12)
    ))
 )

# fig_team.update_layout(
#     updatemenus=[
#         {
#             "buttons": [
#                 {
#                     "label": c,
#                     "method": "update",
#                     "args": [
#                         {
#                             "cells": {
#                                 "values": GP_Race_Wins_Team.T.values
#                                 if c == "All"
#                                 else GP_Race_Wins_Team.loc[GP_Race_Wins_Team["Year"].eq(c)].T.values
#                             }
#                         }
#                     ],
#                 }
#                 for c in ["All"] + GP_Race_Wins_Team["Year"].unique().tolist()
#             ]
#         }
#     ]
# )

fig_team.update_layout(
    title="Race Results All Time By Position and Team",
        title_x=0.5,
        template="plotly_white",

    yaxis = dict(tickfont = dict(size=12)),
    xaxis = dict(tickfont = dict(size=12)),
    font=dict(
        family="PT Sans Narrow",
        size=12,
        color="Black"
    ),
    height=900,
    width=1400,
    title_font_family="PT Sans Narrow",
    margin=dict(l=10, r=10, t=35, b=10),
)
fig_team.show()

In [24]:
fig_team.write_html("Race Results All Time By Position and Team.html")

In [25]:
#Team Sprint Race Results

In [26]:
GP_Sprint_Wins_Team=GP_sprint[GP_sprint["Year"].isin(range(2021,2025))].pivot(index=['Year','GP'], columns=['Position'], values=["TeamName"])
GP_Sprint_Wins_Team.head()

TeamName                   \
Position                                     1                2    
Year GP                                                            
2021 British Grand Prix         Red Bull Racing         Mercedes   
     Italian Grand Prix                Mercedes  Red Bull Racing   
     São Paulo Grand Prix              Mercedes  Red Bull Racing   
2022 Austrian Grand Prix        Red Bull Racing          Ferrari   
     Emilia Romagna Grand Prix  Red Bull Racing          Ferrari   

                                                                  \
Position                                     3                4    
Year GP                                                            
2021 British Grand Prix                Mercedes          Ferrari   
     Italian Grand Prix                 McLaren          McLaren   
     São Paulo Grand Prix               Ferrari  Red Bull Racing   
2022 Austrian Grand Prix                Ferrari         Mercedes   
     Emilia Romagna Grand Prix  Red Bull Racing          Ferrari   

                                                                        \
Position                                     5        6             7    
Year GP                                                                  
2021 British Grand Prix                 McLaren  McLaren        Alpine   
     Italian Grand Prix                Mercedes  Ferrari       Ferrari   
     São Paulo Grand Prix              Mercedes  McLaren       Ferrari   
2022 Austrian Grand Prix        Red Bull Racing   Alpine  Haas F1 Team   
     Emilia Romagna Grand Prix          McLaren  McLaren    Alfa Romeo   

                                                                    \
Position                                       8                9    
Year GP                                                              
2021 British Grand Prix              Aston Martin         Williams   
     Italian Grand Prix         Alfa Romeo Racing  Red Bull Racing   
     São Paulo Grand Prix              AlphaTauri           Alpine   
2022 Austrian Grand Prix                 Mercedes     Haas F1 Team   
     Emilia Romagna Grand Prix       Haas F1 Team           Alpine   

                                                                      \
Position                                  10        11            12   
Year GP                                                                
2021 British Grand Prix               Alpine   Ferrari    AlphaTauri   
     Italian Grand Prix         Aston Martin    Alpine  Aston Martin   
     São Paulo Grand Prix       Aston Martin   McLaren        Alpine   
2022 Austrian Grand Prix          Alfa Romeo   McLaren       McLaren   
     Emilia Romagna Grand Prix  Haas F1 Team  Mercedes    AlphaTauri   

                                                                 \
Position                                       13            14   
Year GP                                                           
2021 British Grand Prix         Alfa Romeo Racing  Aston Martin   
     Italian Grand Prix                    Alpine      Williams   
     São Paulo Grand Prix       Alfa Romeo Racing  Aston Martin   
2022 Austrian Grand Prix             Aston Martin    Alfa Romeo   
     Emilia Romagna Grand Prix       Aston Martin      Mercedes   

                                                                             \
Position                                       15          16            17   
Year GP                                                                       
2021 British Grand Prix         Alfa Romeo Racing  AlphaTauri      Williams   
     Italian Grand Prix                  Williams  AlphaTauri  Haas F1 Team   
     São Paulo Grand Prix              AlphaTauri    Williams      Williams   
2022 Austrian Grand Prix               AlphaTauri    Williams    AlphaTauri   
     Emilia Romagna Grand Prix       Aston Martin      Alpine    AlphaTauri   

                                                

In [27]:
#Remove the column that says full name
GP_Sprint_Wins_Team.columns = GP_Sprint_Wins_Team.columns.droplevel(level=0)

#Reset the index to get a proper df
GP_Sprint_Wins_Team=GP_Sprint_Wins_Team.reset_index()

GP_Sprint_Wins_Team=GP_Sprint_Wins_Team.sort_values(by=["Year","GP"],ascending=[False,True])
GP_Sprint_Wins_Team.head()

Position,Year,GP,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
12,2024,Austrian Grand Prix,Red Bull Racing,McLaren,McLaren,Mercedes,Ferrari,Mercedes,Ferrari,Red Bull Racing,Haas F1 Team,Aston Martin,Alpine,Alpine,RB,RB,Aston Martin,Williams,Williams,Kick Sauber,Haas F1 Team,Kick Sauber
13,2024,Chinese Grand Prix,Red Bull Racing,Mercedes,Red Bull Racing,Ferrari,Ferrari,McLaren,McLaren,Mercedes,Kick Sauber,Haas F1 Team,RB,Kick Sauber,Alpine,Aston Martin,Alpine,RB,Williams,Williams,Haas F1 Team,Aston Martin
14,2024,Miami Grand Prix,Red Bull Racing,Ferrari,Red Bull Racing,RB,Ferrari,McLaren,Haas F1 Team,RB,Alpine,Williams,Kick Sauber,Mercedes,Williams,Kick Sauber,Alpine,Mercedes,Aston Martin,Haas F1 Team,Aston Martin,McLaren
6,2023,Austrian Grand Prix,Red Bull Racing,Red Bull Racing,Ferrari,Aston Martin,Aston Martin,Haas F1 Team,Alpine,Mercedes,McLaren,Mercedes,McLaren,Ferrari,Williams,Haas F1 Team,Alpine,AlphaTauri,AlphaTauri,Williams,Alfa Romeo,Alfa Romeo
7,2023,Azerbaijan Grand Prix,Red Bull Racing,Ferrari,Red Bull Racing,Mercedes,Ferrari,Aston Martin,Mercedes,Aston Martin,Williams,McLaren,Haas F1 Team,Alfa Romeo,Alpine,AlphaTauri,Haas F1 Team,Alfa Romeo,McLaren,Alpine,AlphaTauri,NaN


In [28]:
import plotly.graph_objects as go

fig_sprint_team = go.Figure(go.Table
 (header={"values": GP_Sprint_Wins_Team.columns},
  cells=dict(
    values=GP_Sprint_Wins_Team.T.values,
    # fill_color=[GP_Sprint_Wins_Team.Color],
    align='center', font=dict(color='Black', size=11)
    ))
 )

# fig_sprint_team.update_layout(
#     updatemenus=[
#         {
#             "buttons": [
#                 {
#                     "label": c,
#                     "method": "update",
#                     "args": [
#                         {
#                             "cells": {
#                                 "values": GP_Sprint_Wins_Team.T.values
#                                 if c == "All"
#                                 else GP_Sprint_Wins_Team.loc[GP_Sprint_Wins_Team["Year"].eq(c)].T.values
#                             }
#                         }
#                     ],
#                 }
#                 for c in ["All"] + GP_Sprint_Wins_Team["Year"].unique().tolist()
#             ]
#         }
#     ]
# )

fig_sprint_team.update_layout(
    title="Sprint Race Results All Time By Position and Team",
        title_x=0.5,
        template="plotly_white",

    yaxis = dict(tickfont = dict(size=12)),
    xaxis = dict(tickfont = dict(size=12)),
    font=dict(
        family="PT Sans Narrow",
        size=12,
        color="Black"
    ),
    height=750,
    width=1400,
    title_font_family="PT Sans Narrow",
    margin=dict(l=10, r=10, t=35, b=10),
)
fig_sprint_team.show()

In [29]:
fig_sprint_team.write_html("Sprint Race Results All Time By Position and Team.html")